In [244]:
import pandas as pd
import numpy as np
from scipy.stats import poisson
from scipy.stats import geom
from scipy.stats import binom
from itertools import permutations
import math
from statistics import mean
import random
from scipy.stats import gamma
import matplotlib.pyplot as  plt
import copy
import scipy.stats


In [245]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


#Import Cases, Vaccination and AQI datasets (X dataset)

In [246]:
cases_df=pd.read_csv("/content/gdrive/MyDrive/United_States_COVID-19_Cases_and_Deaths_by_State_over_Time.csv")
vaccine_df=pd.read_csv("/content/gdrive/MyDrive/COVID-19_Vaccinations_in_the_United_States_Jurisdiction.csv")
aqi_2021=pd.read_csv("/content/gdrive/MyDrive/daily_aqi_by_county_2021.csv")
aqi_2020=pd.read_csv("/content/gdrive/MyDrive/daily_aqi_by_county_2020.csv")

In [247]:
cases_df

,submission_date,state,tot_cases,conf_cases,prob_cases,new_case,pnew_case,tot_death,conf_death,prob_death,new_death,pnew_death,created_at,consent_cases,consent_deaths
0,12/01/2021,ND,163565,135705.0,27860.0,589,220.0,1907,NaN,NaN,9,0.0,12/02/2021 02:35:20 PM,Agree,Not agree
1,06/12/2021,AL,545637,421169.0,124468.0,130,51.0,11461,9150.0,2311.0,6,2.0,06/12/2021 12:00:00 AM,Agree,Agree
2,10/30/2021,WA,726837,NaN,NaN,1841,259.0,8628,NaN,NaN,0,0.0,10/30/2021 12:00:00 AM,NaN,NaN
3,03/28/2022,VT,107785,NaN,NaN,467,35.0,585,NaN,NaN,0,0.0,03/29/2022 01:30:11 PM,Not agree,Not agree
4,03/11/2021,MD,390490,NaN,NaN,924,0.0,8549,8345.0,204.0,19,0.0,03/13/2021 12:00:00 AM,NaN,Agree
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50575,12/11/2020,AZ,394804,380243.0,14561.0,6986,459.0,7245,6689.0,556.0,91,12.0,12/12/2020 02:59:37 PM,Agree,Agree
50576,08/13/2021,FSM,7,7.0,0.0,0,0.0,0,0.0,0.0,0,0.0,08/14/2021 02:46:52 PM,Agree,Agree
50577,12/15/2020,DC,25339,NaN,NaN,301,0.0,720,NaN,NaN,4,0.0,12/16/2020 02:28:57 PM,NaN,NaN
50578,07/06/2021,OR,209494,209494.0,0.0,117,0.0,2782,2782.0,0.0,1,0.0,07/07/2021 02:48:49 PM,Agree,Agree


In [248]:
vaccine_df

,Date,MMWR_week,Location,Distributed,Distributed_Janssen,Distributed_Moderna,Distributed_Pfizer,Distributed_Unk_Manuf,Dist_Per_100K,Distributed_Per_100k_12Plus,...,Administered_Dose1_Recip_5PlusPop_Pct,Series_Complete_5Plus,Series_Complete_5PlusPop_Pct,Administered_5Plus,Admin_Per_100k_5Plus,Distributed_Per_100k_5Plus,Series_Complete_Moderna_5Plus,Series_Complete_Pfizer_5Plus,Series_Complete_Janssen_5Plus,Series_Complete_Unk_Manuf_5Plus
0,05/15/2022,20,PR,7552350,215000,2662120,4675230,0,236477,265154,...,95.0,2656609.0,86.4,7210952.0,234410.0,245508.0,955633.0,1561379.0,139478.0,119.0
1,05/15/2022,20,KS,6121515,256400,2354940,3510175,0,210122,249685,...,79.8,1796846.0,65.9,4651023.0,170493.0,224397.0,639434.0,1029902.0,126716.0,794.0
2,05/15/2022,20,VA,19949085,785300,7108700,12055085,0,233718,273014,...,91.3,6263803.0,78.0,16635530.0,207166.0,248431.0,2028444.0,3775026.0,456465.0,3868.0
3,05/15/2022,20,MT,2004895,105200,825200,1074495,0,187588,218494,...,69.3,608133.0,60.4,1602572.0,159045.0,198973.0,238123.0,314353.0,55131.0,526.0
4,05/15/2022,20,IH2,2965895,108400,1311680,1545815,0,142399,0,...,0.0,873824.0,0.0,2255839.0,0.0,0.0,394920.0,440195.0,38703.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33427,12/13/2020,51,AS,3900,0,0,0,0,7003,0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
33428,12/13/2020,51,MP,4875,0,0,0,0,8570,0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
33429,12/13/2020,51,VI,975,0,0,0,0,931,0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
33430,12/13/2020,51,US,13650,0,0,0,0,4,0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN


# Filtering rows to get data for the two assigned states - Delaware and Idaho

In [249]:
cases_df_id_de = cases_df[(cases_df.state=="DE") | (cases_df.state=="ID")]
cases_df_id_de

,submission_date,state,tot_cases,conf_cases,prob_cases,new_case,pnew_case,tot_death,conf_death,prob_death,new_death,pnew_death,created_at,consent_cases,consent_deaths
6,10/06/2021,DE,134690,124148.0,10542.0,444,45.0,1977,1810.0,167.0,2,0.0,10/08/2021 12:00:00 AM,Agree,Agree
18,07/16/2021,DE,109472,102240.0,7232.0,40,14.0,1697,1551.0,146.0,1,0.0,07/18/2021 12:00:00 AM,Agree,Agree
30,10/05/2021,ID,263294,209207.0,54087.0,1835,433.0,2993,2532.0,461.0,11,3.0,10/06/2021 02:06:06 PM,Agree,Agree
38,08/17/2020,ID,27942,26091.0,1851.0,282,32.0,273,245.0,28.0,4,0.0,08/18/2020 02:26:55 PM,Agree,Agree
47,08/05/2021,DE,111201,103716.0,7485.0,182,13.0,1833,1676.0,157.0,0,0.0,08/07/2021 12:00:00 AM,Agree,Agree
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45456,07/02/2021,ID,195172,156211.0,38961.0,83,26.0,2158,1880.0,278.0,4,1.0,07/03/2021 01:44:06 PM,Agree,Agree
45461,05/09/2020,DE,6476,6265.0,211.0,167,5.0,224,213.0,11.0,11,0.0,05/11/2020 12:00:00 AM,Agree,Agree
45462,02/06/2022,ID,391067,306841.0,84226.0,0,0.0,4479,3673.0,806.0,0,0.0,02/07/2022 03:36:42 PM,Agree,Agree
45463,12/04/2021,DE,154934,142206.0,12728.0,761,118.0,2194,2010.0,184.0,0,0.0,12/06/2021 12:00:00 AM,Agree,Agree


#Removing null values

In [250]:
cases_df_id_de = cases_df_id_de.dropna()
cases_df_id_de.isnull().sum()

submission_date    0
state              0
tot_cases          0
conf_cases         0
prob_cases         0
new_case           0
pnew_case          0
tot_death          0
conf_death         0
prob_death         0
new_death          0
pnew_death         0
created_at         0
consent_cases      0
consent_deaths     0
dtype: int64

#Since directly removing the entire row was resulting in a major loss of information, we are filtering just the rows which have NaN value for the column 'Administered'

In [251]:
vaccines_df_id_de=vaccine_df[(vaccine_df.Location=="DE") | (vaccine_df.Location=="ID")]
vaccines_df_id_de=vaccines_df_id_de[vaccines_df_id_de["Administered"].isna()==False]
vaccines_df_id_de

,Date,MMWR_week,Location,Distributed,Distributed_Janssen,Distributed_Moderna,Distributed_Pfizer,Distributed_Unk_Manuf,Dist_Per_100K,Distributed_Per_100k_12Plus,...,Administered_Dose1_Recip_5PlusPop_Pct,Series_Complete_5Plus,Series_Complete_5PlusPop_Pct,Administered_5Plus,Admin_Per_100k_5Plus,Distributed_Per_100k_5Plus,Series_Complete_Moderna_5Plus,Series_Complete_Pfizer_5Plus,Series_Complete_Janssen_5Plus,Series_Complete_Unk_Manuf_5Plus
24,05/15/2022,20,ID,3445790,157600,1351080,1937110,0,192818,230425,...,66.1,976686.0,58.5,2492362.0,149166.0,206228.0,331532.0,562661.0,81633.0,860.0
58,05/15/2022,20,DE,2401655,100100,929800,1371755,0,246636,285977,...,88.3,675725.0,73.5,1833567.0,199508.0,261321.0,236904.0,380865.0,57230.0,726.0
77,05/14/2022,19,ID,3445790,157600,1351080,1937110,0,192818,230425,...,66.0,976005.0,58.4,2487951.0,148902.0,206228.0,331307.0,562224.0,81617.0,857.0
88,05/14/2022,19,DE,2401655,100100,929800,1371755,0,246636,285977,...,88.3,675713.0,73.5,1833525.0,199503.0,261321.0,236900.0,380857.0,57230.0,726.0
141,05/13/2022,19,DE,2398555,100100,929000,1369455,0,246318,285608,...,88.3,675457.0,73.5,1831610.0,199295.0,260983.0,236793.0,380718.0,57220.0,726.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33294,12/16/2020,51,DE,8775,0,0,0,0,901,0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
33310,12/15/2020,51,ID,2925,0,0,0,0,164,0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
33364,12/15/2020,51,DE,975,0,0,0,0,100,0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
33389,12/14/2020,51,DE,975,0,0,0,0,100,0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN


#Sorting the datasets by the date to create daily stats from cumulative stats



In [252]:
cases_df_id_de['submission_date'] = pd.to_datetime(cases_df_id_de['submission_date'])
cases_df_id_de=cases_df_id_de.sort_values(by='submission_date')
cases_df_id_de

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,submission_date,state,tot_cases,conf_cases,prob_cases,new_case,pnew_case,tot_death,conf_death,prob_death,new_death,pnew_death,created_at,consent_cases,consent_deaths
3054,2020-01-22,DE,0,0.0,0.0,0,0.0,0,0.0,0.0,0,0.0,01/24/2020 12:00:00 AM,Agree,Agree
974,2020-01-23,DE,0,0.0,0.0,0,0.0,0,0.0,0.0,0,0.0,01/25/2020 12:00:00 AM,Agree,Agree
7903,2020-01-24,DE,0,0.0,0.0,0,0.0,0,0.0,0.0,0,0.0,01/26/2020 12:00:00 AM,Agree,Agree
533,2020-01-25,DE,0,0.0,0.0,0,0.0,0,0.0,0.0,0,0.0,01/27/2020 12:00:00 AM,Agree,Agree
41165,2020-01-26,DE,0,0.0,0.0,0,0.0,0,0.0,0.0,0,0.0,01/28/2020 12:00:00 AM,Agree,Agree
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3886,2022-05-11,ID,447540,350515.0,97025.0,381,112.0,4930,4017.0,913.0,0,0.0,05/12/2022 01:16:14 PM,Agree,Agree
6263,2022-05-12,ID,447540,350515.0,97025.0,0,0.0,4930,4017.0,913.0,0,0.0,05/13/2022 01:28:57 PM,Agree,Agree
43898,2022-05-12,DE,266673,242758.0,23915.0,966,106.0,2928,2634.0,294.0,0,0.0,05/13/2022 01:28:57 PM,Agree,Agree
6672,2022-05-13,DE,267265,243270.0,23995.0,592,80.0,2931,2637.0,294.0,3,0.0,05/14/2022 01:28:26 PM,Agree,Agree


In [253]:
vaccines_df_id_de['Date'] = pd.to_datetime(vaccines_df_id_de['Date'])
vaccines_df_id_de=vaccines_df_id_de.sort_values(by='Date')
vaccines_df_id_de

,Date,MMWR_week,Location,Distributed,Distributed_Janssen,Distributed_Moderna,Distributed_Pfizer,Distributed_Unk_Manuf,Dist_Per_100K,Distributed_Per_100k_12Plus,...,Administered_Dose1_Recip_5PlusPop_Pct,Series_Complete_5Plus,Series_Complete_5PlusPop_Pct,Administered_5Plus,Admin_Per_100k_5Plus,Distributed_Per_100k_5Plus,Series_Complete_Moderna_5Plus,Series_Complete_Pfizer_5Plus,Series_Complete_Janssen_5Plus,Series_Complete_Unk_Manuf_5Plus
33414,2020-12-14,51,ID,1950,0,0,0,0,109,0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
33389,2020-12-14,51,DE,975,0,0,0,0,100,0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
33310,2020-12-15,51,ID,2925,0,0,0,0,164,0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
33364,2020-12-15,51,DE,975,0,0,0,0,100,0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
33289,2020-12-16,51,ID,2925,0,0,0,0,164,0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141,2022-05-13,19,DE,2398555,100100,929000,1369455,0,246318,285608,...,88.3,675457.0,73.5,1831610.0,199295.0,260983.0,236793.0,380718.0,57220.0,726.0
88,2022-05-14,19,DE,2401655,100100,929800,1371755,0,246636,285977,...,88.3,675713.0,73.5,1833525.0,199503.0,261321.0,236900.0,380857.0,57230.0,726.0
77,2022-05-14,19,ID,3445790,157600,1351080,1937110,0,192818,230425,...,66.0,976005.0,58.4,2487951.0,148902.0,206228.0,331307.0,562224.0,81617.0,857.0
58,2022-05-15,20,DE,2401655,100100,929800,1371755,0,246636,285977,...,88.3,675725.0,73.5,1833567.0,199508.0,261321.0,236904.0,380865.0,57230.0,726.0


In [254]:
vaccines_df_id_de.to_csv('vaccines.csv',index=False)


# We create the daily stats from the cumulative stats by subtracting the previous row from each row for the columns - total cases and total deaths. We then add back the other columns to the dataset.

In [255]:
def change_to_daily(col):
    global cumulative
    temp=col
    col-=cumulative
    cumulative=temp
    return col
        
cases_df_id_de_daily_de=pd.DataFrame()
cases_df_id_de_daily_id=pd.DataFrame()
num_columns=['tot_cases', 
       'tot_death']

for col_name in num_columns:
    cumulative=0
    cases_df_id_de_daily_de[col_name]=cases_df_id_de[col_name][cases_df_id_de.state=="DE"].apply(change_to_daily)
    cumulative=0
    cases_df_id_de_daily_id[col_name]=cases_df_id_de[col_name][cases_df_id_de.state=="ID"].apply(change_to_daily)
cases_df_id_de_daily=pd.concat([cases_df_id_de_daily_de,cases_df_id_de_daily_id])

#We create a new column for both the cases and vaccination dataset and end up with two columns - one with cumulative total for number of cases and deaths and one for the daily number of cases and deaths

In [256]:
vaccine_daily_de=vaccines_df_id_de[vaccines_df_id_de.Location=="DE"]
vaccine_daily_id=vaccines_df_id_de[vaccines_df_id_de.Location=="ID"]


cumulative=0
vaccine_daily_de["Administered_new"]=vaccine_daily_de["Administered"].apply(change_to_daily)
cumulative=0
vaccine_daily_id["Administered_new"]=vaccine_daily_id["Administered"].apply(change_to_daily)
# cases_df_id_de_daily=pd.concat([cases_df_id_de_daily_de,cases_df_id_de_daily_id])
# tot_cases_de
# vaccine_daily_de["Administered_new"].tolist()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [257]:
vaccine_daily_de.to_csv('vaccines_delaware.csv',index=False)
vaccine_daily_id.to_csv('vaccines_idaho.csv',index=False)


In [258]:
rem_cols=['submission_date', 'state',
       'new_case', 'pnew_case',
       'new_death', 'pnew_death', 'created_at', 'consent_cases',
       'consent_deaths','prob_cases','conf_cases', 'conf_death', 'prob_death']
cases_df_id_de_daily_de_final=pd.concat([cases_df_id_de_daily_de,cases_df_id_de[rem_cols][cases_df_id_de.state=="DE"]], axis = 1)
cases_df_id_de_daily_de_final
cases_df_id_de_daily_id_final=pd.concat([cases_df_id_de_daily_id,cases_df_id_de[rem_cols][cases_df_id_de.state=="ID"]], axis = 1)
cases_df_id_de_daily_id_final

,tot_cases,tot_death,submission_date,state,new_case,pnew_case,new_death,pnew_death,created_at,consent_cases,consent_deaths,prob_cases,conf_cases,conf_death,prob_death
45445,1587,41,2020-04-15,ID,101,80.0,2,0.0,04/15/2020 04:22:39 PM,Agree,Agree,80.0,1507.0,41.0,0.0
37699,22,0,2020-04-16,ID,22,1.0,0,14.0,04/16/2020 04:22:39 PM,Agree,Agree,81.0,1528.0,27.0,14.0
5527,46,2,2020-04-17,ID,46,8.0,2,0.0,04/17/2020 04:22:39 PM,Agree,Agree,89.0,1566.0,29.0,14.0
5994,13,1,2020-04-18,ID,13,2.0,1,-14.0,04/18/2020 04:22:39 PM,Agree,Agree,91.0,1577.0,44.0,0.0
41733,4,1,2020-04-19,ID,4,0.0,1,16.0,04/19/2020 04:22:39 PM,Agree,Agree,91.0,1581.0,29.0,16.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3201,303,2,2022-05-09,ID,303,70.0,2,0.0,05/10/2022 01:23:16 PM,Agree,Agree,96913.0,350246.0,4017.0,913.0
673,0,0,2022-05-10,ID,0,0.0,0,0.0,05/11/2022 01:51:16 PM,Agree,Agree,96913.0,350246.0,4017.0,913.0
3886,381,0,2022-05-11,ID,381,112.0,0,0.0,05/12/2022 01:16:14 PM,Agree,Agree,97025.0,350515.0,4017.0,913.0
6263,0,0,2022-05-12,ID,0,0.0,0,0.0,05/13/2022 01:28:57 PM,Agree,Agree,97025.0,350515.0,4017.0,913.0


In [259]:
cases_df_id_de_daily_id_final["tot_cases_old"]=cases_df_id_de["tot_cases"][cases_df_id_de.state=="ID"]
cases_df_id_de_daily_id_final["tot_death_old"]=cases_df_id_de["tot_death"][cases_df_id_de.state=="ID"]
cases_df_id_de_daily_de_final["tot_cases_old"]=cases_df_id_de["tot_cases"][cases_df_id_de.state=="DE"]
cases_df_id_de_daily_de_final["tot_death_old"]=cases_df_id_de["tot_death"][cases_df_id_de.state=="DE"]
cases_df_id_de_daily_id_final

,tot_cases,tot_death,submission_date,state,new_case,pnew_case,new_death,pnew_death,created_at,consent_cases,consent_deaths,prob_cases,conf_cases,conf_death,prob_death,tot_cases_old,tot_death_old
45445,1587,41,2020-04-15,ID,101,80.0,2,0.0,04/15/2020 04:22:39 PM,Agree,Agree,80.0,1507.0,41.0,0.0,1587,41
37699,22,0,2020-04-16,ID,22,1.0,0,14.0,04/16/2020 04:22:39 PM,Agree,Agree,81.0,1528.0,27.0,14.0,1609,41
5527,46,2,2020-04-17,ID,46,8.0,2,0.0,04/17/2020 04:22:39 PM,Agree,Agree,89.0,1566.0,29.0,14.0,1655,43
5994,13,1,2020-04-18,ID,13,2.0,1,-14.0,04/18/2020 04:22:39 PM,Agree,Agree,91.0,1577.0,44.0,0.0,1668,44
41733,4,1,2020-04-19,ID,4,0.0,1,16.0,04/19/2020 04:22:39 PM,Agree,Agree,91.0,1581.0,29.0,16.0,1672,45
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3201,303,2,2022-05-09,ID,303,70.0,2,0.0,05/10/2022 01:23:16 PM,Agree,Agree,96913.0,350246.0,4017.0,913.0,447159,4930
673,0,0,2022-05-10,ID,0,0.0,0,0.0,05/11/2022 01:51:16 PM,Agree,Agree,96913.0,350246.0,4017.0,913.0,447159,4930
3886,381,0,2022-05-11,ID,381,112.0,0,0.0,05/12/2022 01:16:14 PM,Agree,Agree,97025.0,350515.0,4017.0,913.0,447540,4930
6263,0,0,2022-05-12,ID,0,0.0,0,0.0,05/13/2022 01:28:57 PM,Agree,Agree,97025.0,350515.0,4017.0,913.0,447540,4930


#Separating datasets for Idaho and Delaware

In [260]:
vaccines_df_id=vaccines_df_id_de[vaccines_df_id_de.Location=="ID"]
vaccines_df_de=vaccines_df_id_de[vaccines_df_id_de.Location=="DE"]

##Data Cleaning Issues

The data contained missing values. We dealt with this by removing these values.

The format of date was not consistent in the dataset so we use pd.to_datetime() to convert them to the same format.

We sort the data by date in order to covert data from cumulative to daily stats

In [261]:
cases_df_id_de_daily_de_final.to_csv('delaware.csv',index=False)
cases_df_id_de_daily_id_final.to_csv('idaho.csv',index=False)

#Outlier detection using Tukey's rule

In [262]:
def outlier_detection(X, column_name):
  values = sorted(X[column_name])
  Q1 = values[int(np.ceil(len(values) / 4))]
  Q3 = values[int(np.ceil(3 * len(values) / 4))]

  iqr = float(Q3 - Q1)
  outliers = X.loc[(X[column_name] > (Q3 + (1.5 * iqr))) | (X[column_name] < (Q1 - (1.5 * iqr)))]
  return outliers.index


# Performing outlier detection on the Cases dataset



###Since we only perform outlier detection on numerical columns, we create a new list of numerical columns and the perform outlier detection for each column.

In [263]:
column_names = list(cases_df_id_de_daily_de_final.columns)

In [264]:
column_names.remove('submission_date')
column_names.remove('state')
column_names.remove('created_at')
column_names.remove('consent_cases')
column_names.remove('consent_deaths')


In [265]:
cases_df_id_de_daily_de_final

,tot_cases,tot_death,submission_date,state,new_case,pnew_case,new_death,pnew_death,created_at,consent_cases,consent_deaths,prob_cases,conf_cases,conf_death,prob_death,tot_cases_old,tot_death_old
3054,0,0,2020-01-22,DE,0,0.0,0,0.0,01/24/2020 12:00:00 AM,Agree,Agree,0.0,0.0,0.0,0.0,0,0
974,0,0,2020-01-23,DE,0,0.0,0,0.0,01/25/2020 12:00:00 AM,Agree,Agree,0.0,0.0,0.0,0.0,0,0
7903,0,0,2020-01-24,DE,0,0.0,0,0.0,01/26/2020 12:00:00 AM,Agree,Agree,0.0,0.0,0.0,0.0,0,0
533,0,0,2020-01-25,DE,0,0.0,0,0.0,01/27/2020 12:00:00 AM,Agree,Agree,0.0,0.0,0.0,0.0,0,0
41165,0,0,2020-01-26,DE,0,0.0,0,0.0,01/28/2020 12:00:00 AM,Agree,Agree,0.0,0.0,0.0,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38501,1096,0,2022-05-09,DE,1096,166.0,0,0.0,05/10/2022 01:23:16 PM,Agree,Agree,23793.0,241679.0,2634.0,294.0,265472,2928
6554,235,0,2022-05-10,DE,235,16.0,0,0.0,05/11/2022 01:51:16 PM,Agree,Agree,23809.0,241898.0,2634.0,294.0,265707,2928
6349,0,0,2022-05-11,DE,0,0.0,0,0.0,05/12/2022 01:16:14 PM,Agree,Agree,23809.0,241898.0,2634.0,294.0,265707,2928
43898,966,0,2022-05-12,DE,966,106.0,0,0.0,05/13/2022 01:28:57 PM,Agree,Agree,23915.0,242758.0,2634.0,294.0,266673,2928


In [266]:
total_outliers = []
# Outliers for Delaware state features in the Cases dataset
for c in column_names:
  outliers = outlier_detection(cases_df_id_de_daily_de_final, c)
  total_outliers.extend(outliers)
total_outliers = set(total_outliers)
print(len(total_outliers))

188


In [267]:
desired_index = [i for i in cases_df_id_de_daily_de_final.index if i not in total_outliers]
len(desired_index)
cases_df_id_de_daily_de_without_outliers = cases_df_id_de_daily_de_final.loc[desired_index]

In [268]:
cases_df_id_de_daily_de_without_outliers

,tot_cases,tot_death,submission_date,state,new_case,pnew_case,new_death,pnew_death,created_at,consent_cases,consent_deaths,prob_cases,conf_cases,conf_death,prob_death,tot_cases_old,tot_death_old
3054,0,0,2020-01-22,DE,0,0.0,0,0.0,01/24/2020 12:00:00 AM,Agree,Agree,0.0,0.0,0.0,0.0,0,0
974,0,0,2020-01-23,DE,0,0.0,0,0.0,01/25/2020 12:00:00 AM,Agree,Agree,0.0,0.0,0.0,0.0,0,0
7903,0,0,2020-01-24,DE,0,0.0,0,0.0,01/26/2020 12:00:00 AM,Agree,Agree,0.0,0.0,0.0,0.0,0,0
533,0,0,2020-01-25,DE,0,0.0,0,0.0,01/27/2020 12:00:00 AM,Agree,Agree,0.0,0.0,0.0,0.0,0,0
41165,0,0,2020-01-26,DE,0,0.0,0,0.0,01/28/2020 12:00:00 AM,Agree,Agree,0.0,0.0,0.0,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4730,0,0,2022-05-07,DE,0,0.0,0,0.0,05/09/2022 01:14:40 PM,Agree,Agree,23627.0,240749.0,2634.0,294.0,264376,2928
8848,0,0,2022-05-08,DE,0,0.0,0,0.0,05/09/2022 02:22:50 PM,Agree,Agree,23627.0,240749.0,2634.0,294.0,264376,2928
6554,235,0,2022-05-10,DE,235,16.0,0,0.0,05/11/2022 01:51:16 PM,Agree,Agree,23809.0,241898.0,2634.0,294.0,265707,2928
6349,0,0,2022-05-11,DE,0,0.0,0,0.0,05/12/2022 01:16:14 PM,Agree,Agree,23809.0,241898.0,2634.0,294.0,265707,2928


In [269]:
# Outliers for Idaho state features in the Cases dataset
total_outliers = []
for c in column_names:
  outliers = outlier_detection(cases_df_id_de_daily_id_final, c)
  total_outliers.extend(outliers)
total_outliers = set(total_outliers)
print(len(total_outliers))

122


In [270]:
desired_index = [i for i in cases_df_id_de_daily_id_final.index if i not in total_outliers]
len(desired_index)
cases_df_id_de_daily_id_without_outliers = cases_df_id_de_daily_id_final.loc[desired_index]

In [271]:
cases_df_id_de_daily_id_final

,tot_cases,tot_death,submission_date,state,new_case,pnew_case,new_death,pnew_death,created_at,consent_cases,consent_deaths,prob_cases,conf_cases,conf_death,prob_death,tot_cases_old,tot_death_old
45445,1587,41,2020-04-15,ID,101,80.0,2,0.0,04/15/2020 04:22:39 PM,Agree,Agree,80.0,1507.0,41.0,0.0,1587,41
37699,22,0,2020-04-16,ID,22,1.0,0,14.0,04/16/2020 04:22:39 PM,Agree,Agree,81.0,1528.0,27.0,14.0,1609,41
5527,46,2,2020-04-17,ID,46,8.0,2,0.0,04/17/2020 04:22:39 PM,Agree,Agree,89.0,1566.0,29.0,14.0,1655,43
5994,13,1,2020-04-18,ID,13,2.0,1,-14.0,04/18/2020 04:22:39 PM,Agree,Agree,91.0,1577.0,44.0,0.0,1668,44
41733,4,1,2020-04-19,ID,4,0.0,1,16.0,04/19/2020 04:22:39 PM,Agree,Agree,91.0,1581.0,29.0,16.0,1672,45
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3201,303,2,2022-05-09,ID,303,70.0,2,0.0,05/10/2022 01:23:16 PM,Agree,Agree,96913.0,350246.0,4017.0,913.0,447159,4930
673,0,0,2022-05-10,ID,0,0.0,0,0.0,05/11/2022 01:51:16 PM,Agree,Agree,96913.0,350246.0,4017.0,913.0,447159,4930
3886,381,0,2022-05-11,ID,381,112.0,0,0.0,05/12/2022 01:16:14 PM,Agree,Agree,97025.0,350515.0,4017.0,913.0,447540,4930
6263,0,0,2022-05-12,ID,0,0.0,0,0.0,05/13/2022 01:28:57 PM,Agree,Agree,97025.0,350515.0,4017.0,913.0,447540,4930


In [272]:
cases_df_id_de_daily_id_without_outliers

,tot_cases,tot_death,submission_date,state,new_case,pnew_case,new_death,pnew_death,created_at,consent_cases,consent_deaths,prob_cases,conf_cases,conf_death,prob_death,tot_cases_old,tot_death_old
5527,46,2,2020-04-17,ID,46,8.0,2,0.0,04/17/2020 04:22:39 PM,Agree,Agree,89.0,1566.0,29.0,14.0,1655,43
36837,64,3,2020-04-20,ID,64,7.0,3,0.0,04/20/2020 04:22:39 PM,Agree,Agree,98.0,1638.0,32.0,16.0,1736,48
4432,30,3,2020-04-21,ID,30,9.0,3,1.0,04/21/2020 04:22:39 PM,Agree,Agree,107.0,1659.0,34.0,17.0,1766,51
8847,36,3,2020-04-22,ID,36,7.0,3,1.0,04/22/2020 04:22:39 PM,Agree,Agree,114.0,1688.0,36.0,18.0,1802,54
941,34,0,2020-04-23,ID,34,1.0,0,0.0,05/25/2020 03:38:40 PM,Agree,Agree,115.0,1721.0,36.0,18.0,1836,54
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3201,303,2,2022-05-09,ID,303,70.0,2,0.0,05/10/2022 01:23:16 PM,Agree,Agree,96913.0,350246.0,4017.0,913.0,447159,4930
673,0,0,2022-05-10,ID,0,0.0,0,0.0,05/11/2022 01:51:16 PM,Agree,Agree,96913.0,350246.0,4017.0,913.0,447159,4930
3886,381,0,2022-05-11,ID,381,112.0,0,0.0,05/12/2022 01:16:14 PM,Agree,Agree,97025.0,350515.0,4017.0,913.0,447540,4930
6263,0,0,2022-05-12,ID,0,0.0,0,0.0,05/13/2022 01:28:57 PM,Agree,Agree,97025.0,350515.0,4017.0,913.0,447540,4930


In [273]:
cases_df_id_de_daily_de_without_outliers.to_csv('delaware.csv',index=False)
cases_df_id_de_daily_id_without_outliers.to_csv('idaho.csv',index=False)

In [274]:
cases_df_id_de_daily_de_final = cases_df_id_de_daily_de_without_outliers
cases_df_id_de_daily_id_final = cases_df_id_de_daily_id_without_outliers

**Outlier detection Inference for the Cases dataset**

For the Cases dataset, we get 188 rows as outliers from the 843 rows corresponding to the Delaware state.
We get 122 rows as outliers from the 759 rows corresponding to the Idaho state.




## Performing outlier detection for Vaccine dataset

###Since directly applying outlier detection to all columns was resulting in a major loss of information, we are performing the detection on just the columns which are planning to use for our exploratory analysis.

In [275]:
column_names = ['Distributed', 'Administered', 'Series_Complete_Yes']

In [276]:
# Outliers for Idaho state features in the Vaccines dataset
total_outliers = []
for c in column_names:
  outliers = outlier_detection(vaccines_df_id, c)
  total_outliers.extend(outliers)
total_outliers = set(total_outliers)
print(len(total_outliers))

0


In [277]:
desired_index = [i for i in vaccines_df_id.index if i not in total_outliers]
len(desired_index)
vaccines_df_id_without_outliers = vaccines_df_id.loc[desired_index]

In [278]:
vaccines_df_de_without_outliers

,Date,MMWR_week,Location,Distributed,Distributed_Janssen,Distributed_Moderna,Distributed_Pfizer,Distributed_Unk_Manuf,Dist_Per_100K,Distributed_Per_100k_12Plus,...,Administered_Dose1_Recip_5PlusPop_Pct,Series_Complete_5Plus,Series_Complete_5PlusPop_Pct,Administered_5Plus,Admin_Per_100k_5Plus,Distributed_Per_100k_5Plus,Series_Complete_Moderna_5Plus,Series_Complete_Pfizer_5Plus,Series_Complete_Janssen_5Plus,Series_Complete_Unk_Manuf_5Plus
33389,2020-12-14,51,DE,975,0,0,0,0,100,0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
33364,2020-12-15,51,DE,975,0,0,0,0,100,0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
33294,2020-12-16,51,DE,8775,0,0,0,0,901,0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
33221,2020-12-17,51,DE,8775,0,0,0,0,901,0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
33151,2020-12-18,51,DE,8775,0,0,0,0,901,0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
281,2022-05-11,19,DE,2379655,99600,922500,1357555,0,244377,283357,...,88.2,674963.0,73.4,1827864.0,198887.0,258927.0,236630.0,380399.0,57208.0,726.0
230,2022-05-12,19,DE,2388255,100100,928500,1359655,0,245260,284381,...,88.3,675201.0,73.5,1829716.0,199089.0,259863.0,236693.0,380569.0,57214.0,725.0
141,2022-05-13,19,DE,2398555,100100,929000,1369455,0,246318,285608,...,88.3,675457.0,73.5,1831610.0,199295.0,260983.0,236793.0,380718.0,57220.0,726.0
88,2022-05-14,19,DE,2401655,100100,929800,1371755,0,246636,285977,...,88.3,675713.0,73.5,1833525.0,199503.0,261321.0,236900.0,380857.0,57230.0,726.0


In [279]:
# Outliers for Delaware state features in the Vaccines dataset
total_outliers = []
for c in column_names:
  outliers = outlier_detection(vaccines_df_de, c)
  total_outliers.extend(outliers)
total_outliers = set(total_outliers)
print(len(total_outliers))

0


In [280]:
desired_index = [i for i in vaccines_df_de.index if i not in total_outliers]
len(desired_index)
vaccines_df_de_without_outliers = vaccines_df_de.loc[desired_index]

In [281]:
vaccines_df_de_without_outliers

,Date,MMWR_week,Location,Distributed,Distributed_Janssen,Distributed_Moderna,Distributed_Pfizer,Distributed_Unk_Manuf,Dist_Per_100K,Distributed_Per_100k_12Plus,...,Administered_Dose1_Recip_5PlusPop_Pct,Series_Complete_5Plus,Series_Complete_5PlusPop_Pct,Administered_5Plus,Admin_Per_100k_5Plus,Distributed_Per_100k_5Plus,Series_Complete_Moderna_5Plus,Series_Complete_Pfizer_5Plus,Series_Complete_Janssen_5Plus,Series_Complete_Unk_Manuf_5Plus
33389,2020-12-14,51,DE,975,0,0,0,0,100,0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
33364,2020-12-15,51,DE,975,0,0,0,0,100,0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
33294,2020-12-16,51,DE,8775,0,0,0,0,901,0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
33221,2020-12-17,51,DE,8775,0,0,0,0,901,0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
33151,2020-12-18,51,DE,8775,0,0,0,0,901,0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
281,2022-05-11,19,DE,2379655,99600,922500,1357555,0,244377,283357,...,88.2,674963.0,73.4,1827864.0,198887.0,258927.0,236630.0,380399.0,57208.0,726.0
230,2022-05-12,19,DE,2388255,100100,928500,1359655,0,245260,284381,...,88.3,675201.0,73.5,1829716.0,199089.0,259863.0,236693.0,380569.0,57214.0,725.0
141,2022-05-13,19,DE,2398555,100100,929000,1369455,0,246318,285608,...,88.3,675457.0,73.5,1831610.0,199295.0,260983.0,236793.0,380718.0,57220.0,726.0
88,2022-05-14,19,DE,2401655,100100,929800,1371755,0,246636,285977,...,88.3,675713.0,73.5,1833525.0,199503.0,261321.0,236900.0,380857.0,57230.0,726.0


**Outlier detection Inference for the Vaccines dataset**

For the rows corresponding to the Idaho and Delaware state, we get no outliers for the columns mentioned above. 


In [364]:
cases_df_id_de_daily_de_without_outliers.to_csv('delaware_cases.csv',index=False)
cases_df_id_de_daily_id_without_outliers.to_csv('idaho_cases.csv',index=False)
vaccines_df_de_without_outliers.to_csv('delaware_vaccines.csv',index=False)
vaccines_df_id_without_outliers.to_csv('idaho_vaccines.csv',index=False)